# Query TSLA Data from Yahoo Finance

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import timedelta, datetime

#https://mrjbq7.github.io/ta-lib/func.html
from talib import RSI, BBANDS, MACD, MOM, EMA, MA, T3, WMA, BETA, LINEARREG, STDDEV

Here we will request daily data for TSLA and SPY tickers from yahoo finance.

In [2]:
random.seed(123)

# TSLA
tsla = yf.Ticker('TSLA')
tsla_df = tsla.history(period='2y', interval='1d')

# SPY
spy = yf.Ticker('SPY')
spy_df = spy.history(period='2y', interval='1d')

Using the TA-Lib library, we can compute technical indicators to use as features in our forecasting model.

In [3]:
"""TECHNICAL ANALYSIS"""

# MACD
macd, macdsignal, macdhist = MACD(tsla_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Momentum
momentum = MOM(tsla_df['Close'], timeperiod=10)

# RSI
rsi = RSI(tsla_df['Close'], timeperiod=14)

# Bollinger Bands
upperband, middleband, lowerband = BBANDS(tsla_df['Close'], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)

# EMA (Exponential Moving Average)
ema = EMA(tsla_df['Close'], timeperiod=30)

# MA 10 day (Moving Average)
ma10 = MA(tsla_df['Close'], timeperiod=10, matype=0)

# MA 21 day (Moving Average)
ma21 = MA(tsla_df['Close'], timeperiod=21, matype=0)

# MA 50 day (Moving Average)
ma50 = MA(tsla_df['Close'], timeperiod=50, matype=0)

# T3 (Triple Exponential Moving Average)
t3 = T3(tsla_df['Close'], timeperiod=5, vfactor=0)

# WMA (Weighted Moving Average)
wma = WMA(tsla_df['Close'], timeperiod=30)

# Beta
beta = BETA(tsla_df['High'], tsla_df['Low'], timeperiod=5)

# Linear Reg
lm = LINEARREG(tsla_df['Close'], timeperiod=14)

# Standard Deviation
sd = STDDEV(tsla_df['Close'], timeperiod=5, nbdev=1)

In [4]:
# Add TA indicators to TSLA
indicators = [macd, macdsignal, macdhist, momentum, rsi, ema, ma10, ma21, ma50, sd]
names = ['MACD','MACD_SIGNAL','MACD_HIST','MOMENTUM','RSI','EMA','MA10','MA21','MA50','STD_DEV']

for i, name in zip(indicators, names):
    tsla_df = pd.concat((tsla_df, i), axis=1)
    tsla_df = tsla_df.rename(columns={0:name})

In [5]:
# Organize df
tsla_df = tsla_df.rename(columns={'Close':'TSLA'})
tsla_df = tsla_df[['TSLA', 'Volume', 'Dividends', 'Stock Splits',
       'MACD', 'MACD_SIGNAL', 'MACD_HIST', 'MOMENTUM', 'RSI', 'EMA','MA10','MA21','MA50','STD_DEV']]

# Add SPY
tsla_df = tsla_df.merge(spy_df['Close'], how='left', left_index=True, right_index=True).rename(columns={'Close':'SPY'})

In [13]:
# Output data
tsla_df.to_csv('TSLA_raw.csv')